### Import Libraries

In [2]:
from wrapper_functions_CAR import *
tf.config.run_functions_eagerly(True)
np.random.seed(10)

C:\Users\nickl\anaconda3\lib\site-packages\gpflow\experimental\utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(
C:\Users\nickl\anaconda3\lib\site-packages\gpflow\experimental\utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(


TensorFlow version: 2.10.0. Expected: 2.7.0
TensorFlow Probability version: 0.18.0. Expected: 0.15.0


### Loading results and analyzing

In [3]:
#local_dir = 'C:/Users/Admin-Dell/Dropbox/Nick_Cranston/HSPH/Research/Nethery Project/Data/'
local_dir = 'C:/Users/nickl/Dropbox/Nick_Cranston/HSPH/Research/Nethery Project/Data/'
file = local_dir + 'CAR_samples_NY_n10000_realdata_fl64_2models_June282023.pickle'
with open(file, "rb") as input_file:
     #CAR_samples, chain_samples, sampler_stat, mcmc_config, phi_true, u_true, data_sub = pickle.load(input_file)
    res_dict = pickle.load(input_file)
res_dict.keys()

dict_keys(['CAR_samples', 'chain_samples', 'sampler_stat', 'phi_true', 'u_true', 'data', 'adjacency', 'pivot_fit', 'pivot_DGP', 'models', 'mcmc_config'])

## Getting the gradient and likelihood values

In [4]:
adjacency = res_dict['adjacency']
data = res_dict['data']
pivot = res_dict['pivot_fit']
models = res_dict['models']

tau2 = 1
rho = 0.3
print('fixing tau2 and rho')
print('when adding in tau2 and rho, need to update the likelihood function!')

Q = (1/tau2)*(np.diag(adjacency.sum(axis=1)) - rho*adjacency)
Q = tf.constant(Q, dtype = tf.float64)

# define log likelihood function
def target_log_prob_fn_CAR(phi, debug_return = False):
    #Q = (1/tau2)*(np.diag(adjacency.sum(axis=1)) - rho*adjacency)
    #Q = tf.constant(Q, dtype = tf.float64)

    ll = tf.Variable(0., dtype = tf.float64)
    A = tf.Variable(0., dtype = tf.float64)
    B = tf.Variable(0., dtype = tf.float64)
    C = tf.Variable(0., dtype = tf.float64)

    for chain in range(phi.shape[0]):
        # (1) Prob of the CAR random effect values
        A = A - 0.5*tf.reduce_mean(tf.linalg.diag_part(
            tf.linalg.matmul(phi[chain,:,:],tf.linalg.matmul(Q, phi[chain,:,:]), transpose_a = True))) 
    ll = ll + A

    # add in determinant values
    # these are multiplied by the number of chains because they are included in the likelihood for each
    log_det = tf.constant(np.linalg.slogdet(Q)[1], dtype = tf.float64)
    B = 0.5*phi.shape[0]*len(models)*log_det
    #log_det = tf.linalg.logdet(Q)[1], dtype = tf.float64
    ll = ll + B 

    if(pivot == -1):
        # get exponentiated values and sum across models
        exp_phi = tf.math.exp(phi)
        exp_phi_rows = tf.reduce_sum(exp_phi, 2)
    elif(pivot in range(3)):
        phi_np = phi.numpy()
        phi_np = np.insert(phi_np, pivot, 0., axis = 2)
        exp_phi = tf.math.exp(tf.constant(phi_np))
        exp_phi_rows = tf.reduce_sum(exp_phi, 2)
    else:
        raise Exception('Pivot needs to be -1, 0, 1, or 2')

    # get model weights and calculate mean estimate
    u = tf.math.divide(exp_phi, exp_phi_rows[...,None])

    ## Do an (n x m) X (n x m) element wise multiplication
    tmp = tf.math.multiply(data[models].values, u)
    n = tf.reduce_sum(tmp, axis = 2)

    # update the log likelihood 
    C = tf.reduce_sum([tf.math.reduce_sum(
        tf.math.subtract(tf.math.multiply(data['census'],tf.math.log(n[chain,:])),
                         n[chain,:])) for chain in range(phi.shape[0])])
    ll = ll + C

    if debug_return:
        dct = {'phi': A, 
               'det': B,
               'Poisson': C,
               'total': ll}
        return(dct)
    else:
        return(ll) 
    
def target_log_prob_fn_CAR2(phi, debug_return = False):
    #Q = (1/tau2)*(np.diag(adjacency.sum(axis=1)) - rho*adjacency)
    #Q = tf.constant(Q, dtype = tf.float64)

    ll = tf.Variable(0., dtype = tf.float64)
    A = tf.Variable(0., dtype = tf.float64)
    B = tf.Variable(0., dtype = tf.float64)
    C = tf.Variable(0., dtype = tf.float64)

    for chain in range(phi.shape[0]):
        # (1) Prob of the CAR random effect values
        A = A - 0.5*tf.reduce_mean(tf.linalg.diag_part(
            tf.linalg.matmul(phi[chain,:,:],tf.linalg.matmul(Q, phi[chain,:,:]), transpose_a = True))) 
    ll = ll + A

    # add in determinant values
    # these are multiplied by the number of chains because they are included in the likelihood for each
    log_det = tf.linalg.slogdet(Q)[1]
    B = 0.5*phi.shape[0]*len(models)*log_det
    ll = ll + B 

    if(pivot == -1):
        # get exponentiated values and sum across models
        exp_phi = tf.math.exp(phi)
        exp_phi_rows = tf.reduce_sum(exp_phi, 2)
    elif(pivot in range(3)):
        phi_np = phi.numpy()
        phi_np = np.insert(phi_np, pivot, 0., axis = 2)
        exp_phi = tf.math.exp(tf.constant(phi_np))
        exp_phi_rows = tf.reduce_sum(exp_phi, 2)
    else:
        raise Exception('Pivot needs to be -1, 0, 1, or 2')

    # get model weights and calculate mean estimate
    u = tf.math.divide(exp_phi, exp_phi_rows[...,None])

    ## Do an (n x m) X (n x m) element wise multiplication
    tmp = tf.math.multiply(data[models].values, u)
    n = tf.reduce_sum(tmp, axis = 2)

    # update the log likelihood 
    C = tf.reduce_sum([tf.math.reduce_sum(
        tf.math.subtract(tf.math.multiply(data['census'],tf.math.log(n[chain,:])),
                         n[chain,:])) for chain in range(phi.shape[0])])
    ll = ll + C

    if debug_return:
        dct = {'phi': A, 
               'det': B,
               'Poisson': C,
               'total': ll}
        return(dct)
    else:
        return(ll)  

def log_wrapper(phi):
    tt = target_log_prob_fn_CAR(phi)
    return(tt.numpy())

fixing tau2 and rho
when adding in tau2 and rho, need to update the likelihood function!


In [5]:
target_log_prob_fn = get_log_prob_from_results(res_dict)

fixing tau2 and rho
when adding in tau2 and rho, need to update the likelihood function and confirm gradient works!


In [6]:
phi = res_dict['chain_samples'][0]
target_log_prob_fn_CAR(phi), target_log_prob_fn(phi), target_log_prob_fn_CAR2(phi), log_wrapper(phi)

(<tf.Tensor: shape=(), dtype=float64, numpy=1215950602.5388224>,
 <tf.Tensor: shape=(), dtype=float64, numpy=1215950602.5388224>,
 <tf.Tensor: shape=(), dtype=float64, numpy=1215950602.5388224>,
 1215950602.5388224)

In [7]:
grad1, grad2, diff = gradient_comparison(phi, target_log_prob_fn)
grad2[0,:3,:], grad1[0,:3,:]

sum(abs(grad1 - grad2))/sum(abs(grad2)) = 0.001360237670822196


(array([[ 0.23841858, -0.12397766],
        [-0.08106232, -0.19550323],
        [-1.31130219,  0.93221664]]),
 <tf.Tensor: shape=(3, 2), dtype=float64, numpy=
 array([[ 0.23951625, -0.12441089],
        [-0.08057855, -0.19535149],
        [-1.31040306,  0.93256642]])>)

In [9]:
grad1, grad2, diff = gradient_comparison(phi, target_log_prob_fn_CAR2)
grad2[0,:3,:], grad1[0,:3,:], np.sum(abs(diff)), np.sum(abs(grad2))

sum(abs(grad1 - grad2))/sum(abs(grad2)) = 0.001360237670822196


(array([[ 0.23841858, -0.12397766],
        [-0.08106232, -0.19550323],
        [-1.31130219,  0.93221664]]),
 <tf.Tensor: shape=(3, 2), dtype=float64, numpy=
 array([[ 0.23951625, -0.12441089],
        [-0.08057855, -0.19535149],
        [-1.31040306,  0.93256642]])>,
 0.5694585164372489,
 418.64633560180664)